In [21]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect('example.db')
def rsq(sql):
    df = pd.read_sql(sql, conn)
    return df

In [22]:
# conn1v = sqlite3.connect('database.sqlite')
# pd.read_sql("SELECT * FROM sqlite_master ", conn1v)   
# def rsqsq(sql):
#     df = pd.read_sql(sql, conn1v)
#     return df 

In [23]:
# df = pd.read_sql("SELECT * FROM Match ", conn1v)
# df.to_sql('match', conn, if_exists='replace',index=False)
# df[['id', 'country_id', 'season', 'stage', 'date', 'hometeam_id',
#        'awayteam_id', 'home_goal', 'away_goal']]

In [24]:
rsq("""SELECT 
    -- Select the id, country name, season, home, and away goals
    m.id,
    c.name AS country,
    m.season,
    m.home_goal,
    m.away_goal,
    -- Use a window to include the aggregate average in each row
    AVG(m.home_goal + m.away_goal) OVER() AS overall_avg
FROM match AS m
LEFT JOIN country AS c ON m.country_id = c.id;""")

,id,country,season,home_goal,away_goal,overall_avg
0,8727,Germany,2011/2012,3,1.0,2.874805
1,8728,Germany,2011/2012,2,0.0,2.874805
2,8729,Germany,2011/2012,2,1.0,2.874805
3,8730,Germany,2011/2012,2,0.0,2.874805
4,8731,Germany,2011/2012,3,0.0,2.874805
...,...,...,...,...,...,...
3208,9422,Germany,2013/2014,1,2.0,2.874805
3209,9423,Germany,2013/2014,1,3.0,2.874805
3210,9424,Germany,2013/2014,0,0.0,2.874805
3211,9425,Germany,2013/2014,4,0.0,2.874805


In [25]:
rsq("""SELECT 
    -- Select the league name and average goals scored
    l.name AS league,
    AVG(m.home_goal + m.away_goal) AS avg_goals,
    -- Rank each league according to the average goals
    RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal)) AS league_rank
FROM league AS l
LEFT JOIN match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
-- Order the query by the rank you created
ORDER BY league_rank;""")

,league,avg_goals,league_rank
0,Italy Serie A,2.436709,1
1,Spain LIGA BBVA,2.763158,2
2,Germany 1. Bundesliga,2.807882,3
3,England Premier League,10.000000,4
4,Netherlands Eredivisie,10.000000,4


In [26]:
rsq("""SELECT 
    -- Select the league name and average goals scored
    l.name AS league,
    AVG(m.home_goal + m.away_goal) AS avg_goals,
    -- Rank leagues in descending order by average goals
    RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal) DESC) AS league_rank
FROM league AS l
LEFT JOIN match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
-- Order the query by the rank you created
ORDER BY league_rank;""")

,league,avg_goals,league_rank
0,Netherlands Eredivisie,10.000000,1
1,England Premier League,10.000000,1
2,Germany 1. Bundesliga,2.807882,3
3,Spain LIGA BBVA,2.763158,4
4,Italy Serie A,2.436709,5


In [39]:
# df = rsq("""SELECT * from match limit 500""")
# df['hometeam_id'] = 8673
# df['awayteam_id'] = 8673
# df.to_sql('match', conn, if_exists='append',index=False)

500

In [40]:
rsq("""SELECT 
    date,
    season,
    home_goal,
    away_goal,
    CASE WHEN hometeam_id = 8673 THEN 'home' 
         ELSE 'away' END AS warsaw_location,
    -- Calculate the average goals scored partitioned by season
    AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,
    AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg
FROM match
-- Filter the data set for Legia Warszawa matches only
WHERE 
    hometeam_id = 8673 
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;""")

,date,season,home_goal,away_goal,warsaw_location,season_homeavg,season_awayavg
0,2013-01-18,2012/2013,5,4.0,home,1.611650,1.339806
1,2013-01-18,2012/2013,5,4.0,home,1.611650,1.339806
2,2013-01-18,2012/2013,5,4.0,home,1.611650,1.339806
3,2012-03-10,2011/2012,7,1.0,home,1.640777,1.165049
4,2012-03-30,2011/2012,4,4.0,home,1.640777,1.165049
...,...,...,...,...,...,...,...
1495,2013-11-08,2013/2014,0,0.0,home,1.715909,1.511364
1496,2013-11-30,2013/2014,0,0.0,home,1.715909,1.511364
1497,2013-12-14,2013/2014,0,0.0,home,1.715909,1.511364
1498,2013-12-21,2013/2014,0,0.0,home,1.715909,1.511364


In [41]:
rsq("""SELECT 
    date,
    season,
    home_goal,
    away_goal,
    CASE WHEN hometeam_id = 8673 THEN 'home' 
         ELSE 'away' END AS warsaw_location,
    -- Calculate average goals partitioned by season and month
    AVG(home_goal) OVER(PARTITION BY season, 
            cast(strftime('%m', date) as int)) AS season_mo_home,
    AVG(away_goal) OVER(PARTITION BY season, 
            cast(strftime('%m', date) as int)) AS season_mo_away
--    # AVG(home_goal) OVER(PARTITION BY season, 
--    #      	EXTRACT(MONTH FROM date)) AS season_mo_home,
--    # AVG(away_goal) OVER(PARTITION BY season, 
--    #         EXTRACT(MONTH FROM date)) AS season_mo_away
FROM match
WHERE 
    hometeam_id = 8673 
    OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC;""")

,date,season,home_goal,away_goal,warsaw_location,season_mo_home,season_mo_away
0,2013-01-18,2012/2013,5,4.0,home,1.307692,1.307692
1,2013-01-18,2012/2013,5,4.0,home,1.307692,1.307692
2,2013-01-18,2012/2013,5,4.0,home,1.307692,1.307692
3,2012-03-10,2011/2012,7,1.0,home,1.441860,1.534884
4,2012-03-30,2011/2012,4,4.0,home,1.441860,1.534884
...,...,...,...,...,...,...,...
1495,2013-12-21,2013/2014,0,0.0,home,1.464286,1.428571
1496,2013-12-14,2013/2014,0,0.0,home,1.464286,1.428571
1497,2013-12-21,2013/2014,0,0.0,home,1.464286,1.428571
1498,2013-12-14,2013/2014,0,0.0,home,1.464286,1.428571


In [30]:
rsq("""SELECT 
    date,
    home_goal,
    away_goal,
    -- Create a running total and running average of home goals
    SUM(home_goal) OVER(ORDER BY date 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,
    AVG(home_goal) OVER(ORDER BY date 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_avg
FROM match
WHERE 
    hometeam_id = 9908 
    AND season = '2011/2012';""")

,date,home_goal,away_goal,running_total,running_avg
0,2011-08-05,3,1.0,3,3.000000
1,2011-08-06,2,0.0,5,2.500000
2,2011-08-06,2,1.0,7,2.333333
3,2011-08-06,3,0.0,10,2.500000
4,2011-08-06,0,3.0,10,2.000000
...,...,...,...,...,...
296,2013-01-20,1,1.0,487,1.639731
297,2013-01-26,0,0.0,487,1.634228
298,2013-01-26,0,0.0,487,1.628763
299,2013-01-26,2,1.0,489,1.630000


In [31]:
rsq("""SELECT 
    -- Select the date, home goal, and away goals
    date,
    home_goal,
    away_goal,
    -- Create a running total and running average of home goals
    SUM(home_goal) OVER(ORDER BY date DESC
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_total,
    AVG(home_goal) OVER(ORDER BY date DESC
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_avg
FROM match
WHERE 
    awayteam_id = 9908 
    AND season = '2011/2012';""")

,date,home_goal,away_goal,running_total,running_avg
0,2013-01-27,0,2.0,483,1.610000
1,2013-01-26,0,0.0,483,1.615385
2,2013-01-26,0,0.0,483,1.620805
3,2013-01-26,2,1.0,483,1.626263
4,2013-01-20,2,3.0,481,1.625000
...,...,...,...,...,...
295,2011-08-06,3,0.0,8,1.600000
296,2011-08-06,0,3.0,5,1.250000
297,2011-08-06,2,2.0,5,1.666667
298,2011-08-06,0,1.0,3,1.500000


In [42]:
# df = rsq("""SELECT * from match limit 300""")
# df['awayteam_id'] = 9908
# df['season'] = '2011/2012'
# df.to_sql('match', conn, if_exists='append',index=False)

300

In [43]:
# df = rsq("""SELECT 
#     t.*
# FROM match AS m
# -- Left join team on the home team ID and team API id
# LEFT JOIN team AS t 
# ON m.hometeam_id = t.team_api_id
# WHERE 
#     -- Filter for 2014/2015 and Manchester United as the home team
#     m.season = '2014/2015'
#     """)
# df['team_long_name'] = 'Manchester United'
# df.to_sql('team', conn, if_exists='append',index=False)

209

In [45]:
rsq("""SELECT 
    m.id, 
    t.team_long_name,
    -- Identify matches as home/away wins or ties
    CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
         WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
         ELSE 'Tie' END AS outcome
FROM match AS m
-- Left join team on the home team ID and team API id
LEFT JOIN team AS t 
ON m.hometeam_id = t.team_api_id
WHERE 
    -- Filter for 2014/2015 and Manchester United as the home team
    m.season = '2014/2015'
    AND t.team_long_name = 'Manchester United';""")

,id,team_long_name,outcome
0,9645,Manchester United,MU Win
1,9645,Manchester United,MU Win
2,9645,Manchester United,MU Win
3,9645,Manchester United,MU Win
4,9645,Manchester United,MU Win
...,...,...,...
2370,24016,Manchester United,MU Win
2371,24016,Manchester United,MU Win
2372,24114,Manchester United,MU Win
2373,24114,Manchester United,MU Win


In [46]:
rsq("""SELECT 
    m.id, 
    t.team_long_name,
    -- Identify matches as home/away wins or ties
    CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
         WHEN m.home_goal < m.away_goal THEN 'MU Win' 
         ELSE 'Tie' END AS outcome
-- Join team table to the match table
FROM match AS m
LEFT JOIN team AS t 
ON m.awayteam_id = t.team_api_id
WHERE 
    -- Filter for 2014/2015 and Manchester United as the away team
    m.season = '2014/2015'
    AND t.team_long_name = 'Manchester United';""")

,id,team_long_name,outcome
0,9645,Manchester United,MU Loss
1,9645,Manchester United,MU Loss
2,9645,Manchester United,MU Loss
3,9645,Manchester United,MU Loss
4,9645,Manchester United,MU Loss
...,...,...,...
2349,9950,Manchester United,MU Win
2350,9950,Manchester United,MU Win
2351,9950,Manchester United,MU Win
2352,24123,Manchester United,Tie


In [47]:
rsq("""-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select team names, the date and goals
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN away ON m.id = away.id
WHERE m.season = '2014/2015'
      AND (home.team_long_name = 'Manchester United' 
           OR away.team_long_name = 'Manchester United');""")

,date,home_team,away_team,home_goal,away_goal
0,2014-08-22,FC Bayern Munich,Manchester United,2,1.0
1,2014-08-22,Manchester United,Manchester United,2,1.0
2,2014-08-22,Manchester United,VfL Wolfsburg,2,1.0
3,2014-08-23,Borussia Dortmund,Manchester United,0,2.0
4,2014-08-23,Manchester United,Bayer 04 Leverkusen,0,2.0
...,...,...,...,...,...
596,2015-04-05,Manchester United,Granada CF,9,1.0
597,2015-05-23,Manchester United,Getafe CF,7,3.0
598,2014-09-20,Manchester United,Manchester United,2,NaN
599,2014-09-20,Manchester United,Real Madrid CF,2,NaN


In [48]:
rsq("""-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select columns and and rank the matches by goal difference
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal,
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN AWAY ON m.id = away.id
WHERE m.season = '2014/2015'
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'));""")

,date,home_team,away_team,home_goal,away_goal,match_rank
0,2015-02-14,FC Bayern Munich,Manchester United,8,0.0,1
1,2015-02-14,Manchester United,Manchester United,8,0.0,1
2,2015-02-21,Manchester United,FC Bayern Munich,0,6.0,133
3,2015-02-21,Manchester United,Manchester United,0,6.0,133
4,2014-10-18,FC Bayern Munich,Manchester United,6,0.0,133
...,...,...,...,...,...,...
284,2014-10-25,TSG 1899 Hoffenheim,Manchester United,1,0.0,11255
285,2014-10-24,Manchester United,1. FC Köln,0,1.0,11255
286,2014-10-24,Manchester United,Manchester United,0,1.0,11255
287,2014-10-25,Manchester United,Manchester United,4,5.0,11255


In [49]:
rsq("""-- Set up the home team CTE
WITH home AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Win'
           WHEN m.home_goal < m.away_goal THEN 'MU Loss' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
away AS (
  SELECT m.id, t.team_long_name,
      CASE WHEN m.home_goal > m.away_goal THEN 'MU Loss'
           WHEN m.home_goal < m.away_goal THEN 'MU Win' 
           ELSE 'Tie' END AS outcome
  FROM match AS m
  LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select columns and and rank the matches by goal difference
SELECT DISTINCT
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal, m.away_goal,
    RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM match AS m
LEFT JOIN home ON m.id = home.id
LEFT JOIN AWAY ON m.id = away.id
WHERE m.season = '2014/2015'
      AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU Loss')
      OR (away.team_long_name = 'Manchester United' AND away.outcome = 'MU Loss'));""")

,date,home_team,away_team,home_goal,away_goal,match_rank
0,2015-02-14,FC Bayern Munich,Manchester United,8,0.0,1
1,2015-02-14,Manchester United,Manchester United,8,0.0,1
2,2015-02-21,Manchester United,FC Bayern Munich,0,6.0,133
3,2015-02-21,Manchester United,Manchester United,0,6.0,133
4,2014-10-18,FC Bayern Munich,Manchester United,6,0.0,133
...,...,...,...,...,...,...
284,2014-10-25,TSG 1899 Hoffenheim,Manchester United,1,0.0,11255
285,2014-10-24,Manchester United,1. FC Köln,0,1.0,11255
286,2014-10-24,Manchester United,Manchester United,0,1.0,11255
287,2014-10-25,Manchester United,Manchester United,4,5.0,11255
